# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [22]:
# Define the target set y using the credit_ranking column
y=loans_df['credit_ranking']

# Display a sample of y
display(y.head())

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [23]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns='credit_ranking')

# Review the features DataFrame
display(X.head())

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [24]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [26]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled)

array([[-0.56794375, -1.18088385, -0.19241296, ...,  0.73249574,
        -0.22553894, -0.31915735],
       [-0.00940983,  0.80065564, -0.04123135, ..., -0.03016065,
        -0.75844691, -1.15783431],
       [ 2.33643265, -0.84597577,  1.67216021, ..., -1.36480934,
         0.42579302,  0.05358797],
       ...,
       [-0.9589175 ,  1.07974571, -0.89792713, ...,  0.22405814,
        -0.9952949 ,  0.33314696],
       [-0.62379715,  0.54947458, -1.35147195, ...,  0.35116754,
        -0.46238693, -1.34420697],
       [ 0.43741731, -0.73433974,  1.16822152, ..., -0.66570765,
         0.18894504,  1.91731456]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [30]:
# Define the the number of inputs (features) to the model

number_input_features = len(X.columns)

# Review the number of features
display(number_input_features)


11

In [31]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 8

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 5

# Define the number of neurons in the output layer
output_nodes = 1


In [32]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu', input_dim=number_input_features))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147 (588.00 B)

 Trainable params: 147 (588.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [34]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step - accuracy: 0.5360 - loss: 0.7036
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - accuracy: 0.5394 - loss: 0.6816
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 335us/step - accuracy: 0.5554 - loss: 0.6682
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step - accuracy: 0.5821 - loss: 0.6451
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - accuracy: 0.6327 - loss: 0.6226
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 307us/step - accuracy: 0.6764 - loss: 0.6125
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7070 - loss: 0.5898
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step - accuracy: 0.7158 - loss: 0.5805
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - accuracy: 0.7084 - loss: 0.5699
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 316us/step - accuracy: 0.7426 - loss: 0.5492
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step - accuracy: 0.7235 - loss: 0.5486
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step -

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [36]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 4ms/step - accuracy: 0.7500 - loss: 0.5085
Loss: 0.5084982514381409, Accuracy: 0.75


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [39]:
# Set the model's file path
file_path = Path("saved_models/student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [40]:
# Set the model's file path
file_path = Path('saved_models/student_loans.keras')

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [41]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions

13/13 - 0s - 3ms/step


array([[0.37625384],
       [0.30179626],
       [0.7864264 ],
       [0.70910144],
       [0.9874052 ],
       [0.9322934 ],
       [0.96337754],
       [0.14842641],
       [0.6142664 ],
       [0.267942  ],
       [0.8977447 ],
       [0.25468877],
       [0.5020932 ],
       [0.9762608 ],
       [0.5815085 ],
       [0.39579403],
       [0.9156081 ],
       [0.32907876],
       [0.4285326 ],
       [0.38845754],
       [0.6066249 ],
       [0.87710464],
       [0.2520315 ],
       [0.8874036 ],
       [0.31578222],
       [0.9604118 ],
       [0.8667744 ],
       [0.5819694 ],
       [0.2564709 ],
       [0.27887234],
       [0.58763444],
       [0.9722351 ],
       [0.22190374],
       [0.9577397 ],
       [0.12757869],
       [0.61081845],
       [0.22896773],
       [0.41374913],
       [0.97333354],
       [0.22877403],
       [0.94984585],
       [0.11420981],
       [0.08010884],
       [0.95384336],
       [0.0833165 ],
       [0.6833359 ],
       [0.17474061],
       [0.301

In [42]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [43]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.72      0.76      0.74       188
           1       0.77      0.75      0.76       212

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

